In [5]:
!pip install uszipcode
!pip install censusgeocode
!pip install geopy
!pip install pgeocode

import numpy as np
import pandas as pd
import pyspark as ps# for the pyspark suite
import helper9 as h

spark = ps.sql.SparkSession.builder \
            .master("local[4]") \
            .appName("case study") \
            .getOrCreate()

sc = spark.sparkContext 

In [6]:
rdd = sc.textFile('reservations2018.csv')\
        .map(h.parse_rows)\
        .map(h.caster)\
        .map(h.state_filter)\
        .map(h.cust_country_filter)\
        .map(h.clean_zips)\
        .map(h.five_zips)\
        .map(h.add_coords)\
        .map(h.distance)\
        

    

In [7]:
rdd.count()

3424272

In [11]:
from pyspark.sql.types import *
res_schema = StructType([
    StructField('Res_ID', StringType(),True),
    StructField('Location', StringType(), True),
    StructField('Park', StringType(),True),
    StructField('SiteType', StringType(),True), 
    StructField('UseType', StringType(),True),
    StructField('FacState', StringType(),True),
    StructField('FacLong', FloatType(),True),
    StructField('FacLat', FloatType(),True),
    StructField('CustZIP', StringType(),True),
    StructField('CustCountry', StringType(),True),
    StructField('CustSize', IntegerType(),True),
    StructField('CustState', StringType(), True),
    StructField('CustLat', FloatType(),True),
    StructField('CustLong', FloatType(),True),
    StructField('Dist', FloatType(), True)])

In [12]:
res_df = spark.createDataFrame(rdd.filter(bool), res_schema)
res_df.show(2)

+----------+-----------------+--------------------+--------------------+---------+--------+-----------+-------+-------+-----------+--------+---------+-------+--------+---------+
|    Res_ID|         Location|                Park|            SiteType|  UseType|FacState|    FacLong| FacLat|CustZIP|CustCountry|CustSize|CustState|CustLat|CustLong|     Dist|
+----------+-----------------+--------------------+--------------------+---------+--------+-----------+-------+-------+-----------+--------+---------+-------+--------+---------+
|3034963543|RIO GRANDE NF- FS|Aspen Glade --- F...|STANDARD NONELECTRIC|Overnight|      CO|-106.273056|37.0725|  80923|        USA|       2|       CO|  38.92| -104.71|246.77235|
|3051273848|RIO GRANDE NF- FS|Aspen Glade --- F...|STANDARD NONELECTRIC|Overnight|      CO|-106.273056|37.0725|  87015|        USA|       8|       NM|   35.1|  -106.2| 218.9687|
+----------+-----------------+--------------------+--------------------+---------+--------+-----------+-------

In [13]:
res_df1 = res_df.toPandas()
res_df1

,Res_ID,Location,Park,SiteType,UseType,FacState,FacLong,FacLat,CustZIP,CustCountry,CustSize,CustState,CustLat,CustLong,Dist
0,3034963543,RIO GRANDE NF- FS,Aspen Glade --- FPIN24,STANDARD NONELECTRIC,Overnight,CO,-106.273056,37.072498,80923,USA,2,CO,38.919998,-104.709999,246.772354
1,3051273848,RIO GRANDE NF- FS,Aspen Glade --- FPIN24,STANDARD NONELECTRIC,Overnight,CO,-106.273056,37.072498,87015,USA,8,NM,35.099998,-106.199997,218.968704
2,3051863567,RIO GRANDE NF- FS,Aspen Glade --- FPIN24,STANDARD NONELECTRIC,Overnight,CO,-106.273056,37.072498,80021,USA,2,CO,39.900002,-105.120003,329.588440
3,3063451302,RIO GRANDE NF- FS,Aspen Glade --- FPIN24,STANDARD NONELECTRIC,Overnight,CO,-106.273056,37.072498,75116,USA,2,TX,32.660000,-96.919998,984.677185
4,3068573240,RIO GRANDE NF- FS,Aspen Glade --- FPIN24,STANDARD NONELECTRIC,Overnight,CO,-106.273056,37.072498,55447,USA,3,MN,45.009998,-93.489998,1387.145630
5,3069995240,RIO GRANDE NF- FS,Aspen Glade --- FPIN24,STANDARD NONELECTRIC,Overnight,CO,-106.273056,37.072498,74033,USA,3,OK,35.939999,-96.019997,926.596313
6,3084570686,RIO GRANDE NF- FS,Aspen Glade --- FPIN24,STANDARD NONELECTRIC,Overnight,CO,-106.273056,37.072498,87122,USA,5,NM,35.200001,-106.500000,208.776978
7,3085291069,RIO GRANDE NF- FS,Aspen Glade --- FPIN24,STANDARD NONELECTRIC,Overnight,CO,-106.273056,37.072498,87123,USA,4,NM,35.049999,-106.480003,225.189896
8,3092704335,RIO GRANDE NF- FS,Aspen Glade --- FPIN24,STANDARD NONELECTRIC,Overnight,CO,-106.273056,37.072498,78748,USA,2,TX,30.160000,-97.820000,1095.940308
9,3097436816,RIO GRANDE NF- FS,Aspen Glade --- FPIN24,STANDARD NONELECTRIC,Overnight,CO,-106.273056,37.072498,80231,USA,2,CO,39.669998,-104.889999,312.626587


In [14]:
res_df1.to_csv('co_sites.csv', index=False)